In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [5]:
from huggingface_hub import HfApi, create_repo

api = HfApi()
create_repo(repo_id="YoussefTolba/td3-lunarlander-v3", exist_ok=True)


RepoUrl('https://huggingface.co/YoussefTolba/td3-lunarlander-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='YoussefTolba/td3-lunarlander-v3')

# TD3 model

In [6]:
# 1. Install system-level dependencies (Fixes the box2d build error)
!sudo apt-get update
!sudo apt-get install -y swig

# 2. Install Python libraries
# We force a reinstall of gymnasium to ensure box2d links correctly
!pip install gymnasium[box2d] moviepy imageio wandb huggingface_hub

# 3. (Optional) Fix specific Colab rendering issues
!pip install imageio-ffmpeg

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

##  LunarLander-v3

In [7]:
import os
import random
import time
import uuid
from dataclasses import dataclass, asdict
from typing import Optional

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from huggingface_hub import HfApi, create_repo, upload_folder

# ==========================================
# 1. Configuration & Hyperparameters
# ==========================================
@dataclass
class TD3Config:
    # Experiment Settings
    env_id: str = "LunarLander-v3"
    project_name: str = "td3-lunarlander-continuous"
    run_name: str = f"td3_run_{str(uuid.uuid4())[:8]}"
    seed: int = 42

    # Training Duration
    total_timesteps: int = 100_000
    learning_starts: int = 10_000

    # Hyperparameters
    hidden_dim: int = 256
    actor_lr: float = 3e-4
    critic_lr: float = 3e-4
    batch_size: int = 256
    buffer_size: int = 1_000_000
    gamma: float = 0.99
    tau: float = 0.005

    # TD3 Specifics
    policy_noise: float = 0.2
    noise_clip: float = 0.5
    policy_delay: int = 2
    exploration_noise: float = 0.1

    # Logging & Saving
    eval_freq: int = 5_000
    save_model: bool = True
    hf_repo_id: str = "YoussefTolba/td3-lunarlander-v3"

# ==========================================
# 2. Replay Buffer
# ==========================================
class ReplayBuffer:
    def __init__(self, state_dim, action_dim, max_size=1e6):
        self.max_size = int(max_size)
        self.ptr = 0
        self.size = 0
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.state = np.zeros((self.max_size, state_dim))
        self.action = np.zeros((self.max_size, action_dim))
        self.next_state = np.zeros((self.max_size, state_dim))
        self.reward = np.zeros((self.max_size, 1))
        self.not_done = np.zeros((self.max_size, 1))

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        return (
            torch.FloatTensor(self.state[ind]).to(self.device),
            torch.FloatTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind]).to(self.device),
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )

# ==========================================
# 3. Neural Networks
# ==========================================
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action, hidden_dim=256):
        super(Actor, self).__init__()
        self.l1 = nn.Linear(state_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, action_dim)
        self.max_action = max_action

    def forward(self, state):
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a))

class Critic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super(Critic, self).__init__()
        self.l1 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, 1)

        self.l4 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.l5 = nn.Linear(hidden_dim, hidden_dim)
        self.l6 = nn.Linear(hidden_dim, 1)

    def forward(self, state, action):
        sa = torch.cat([state, action], 1)
        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1)
        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

# ==========================================
# 4. TD3 Algorithm
# ==========================================
class TD3:
    def __init__(self, state_dim, action_dim, max_action, config: TD3Config):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conf = config
        self.max_action = max_action
        self.total_it = 0

        self.actor = Actor(state_dim, action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target = Actor(state_dim, action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=config.actor_lr)

        self.critic = Critic(state_dim, action_dim, config.hidden_dim).to(self.device)
        self.critic_target = Critic(state_dim, action_dim, config.hidden_dim).to(self.device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=config.critic_lr)

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(self.device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer):
        self.total_it += 1

        # Sample replay buffer
        state, action, next_state, reward, not_done = replay_buffer.sample(self.conf.batch_size)

        with torch.no_grad():
            noise = (torch.randn_like(action) * self.conf.policy_noise).clamp(-self.conf.noise_clip, self.conf.noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + not_done * self.conf.gamma * target_Q

        current_Q1, current_Q2 = self.critic(state, action)
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        actor_loss = None
        if self.total_it % self.conf.policy_delay == 0:
            actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)
            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)

        return critic_loss.item(), (actor_loss.item() if actor_loss else None)

    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic.pth")
        torch.save(self.actor.state_dict(), filename + "_actor.pth")

# ==========================================
# 5. Helpers: Evaluation & Recording
# ==========================================
def evaluate_and_record(policy, env_id, seed, step, video_folder="videos"):
    """
    Evaluates the agent for 5 episodes.
    Records the FIRST episode of the evaluation.
    """
    eval_env = gym.make(env_id, continuous=True, render_mode="rgb_array")

    # A custom prefix to identify the step
    video_prefix = f"td3-step-{step}"

    eval_env = gym.wrappers.RecordVideo(
        eval_env,
        video_folder=video_folder,
        name_prefix=video_prefix,
        episode_trigger=lambda x: x == 0, # Record only the first episode in this call
        disable_logger=True
    )

    rewards = []
    video_path = None

    for i in range(5):
        state, _ = eval_env.reset(seed=seed + 100 + i)
        terminated, truncated = False, False
        episode_reward = 0

        while not (terminated or truncated):
            action = policy.select_action(np.array(state))
            state, reward, terminated, truncated, _ = eval_env.step(action)
            episode_reward += reward

        rewards.append(episode_reward)

    eval_env.close()

    # Format: {prefix}-episode-0.mp4
    expected_path = os.path.join(video_folder, f"{video_prefix}-episode-0.mp4")
    if os.path.exists(expected_path):
        video_path = expected_path

    return np.mean(rewards), video_path

# ==========================================
# 6. Main Training Loop
# ==========================================
def run_training():
    conf = TD3Config()

    # === FIXED WANDB INIT ===
    # monitor_gym=False prevents the AttributeError with RecordVideo
    wandb.init(
        project=conf.project_name,
        name=conf.run_name,
        config=asdict(conf),
        monitor_gym=False,
        save_code=True
    )

    env = gym.make(conf.env_id, continuous=True)
    env.action_space.seed(conf.seed)
    torch.manual_seed(conf.seed)
    np.random.seed(conf.seed)

    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])

    replay_buffer = ReplayBuffer(state_dim, action_dim, conf.buffer_size)
    policy = TD3(state_dim, action_dim, max_action, conf)

    state, _ = env.reset(seed=conf.seed)
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0

    print(f"---------------------------------------")
    print(f"Starting Training: {conf.env_id} | TD3 | Seed: {conf.seed}")
    print(f"---------------------------------------")

    for t in range(int(conf.total_timesteps)):
        episode_timesteps += 1

        if t < conf.learning_starts:
            action = env.action_space.sample()
        else:
            action = (
                policy.select_action(np.array(state))
                + np.random.normal(0, max_action * conf.exploration_noise, size=action_dim)
            ).clip(-max_action, max_action)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        replay_buffer.add(state, action, next_state, reward, float(terminated))
        state = next_state
        episode_reward += reward

        if t >= conf.learning_starts:
            critic_loss, actor_loss = policy.train(replay_buffer)

            if t % 100 == 0:
                logs = {"train/critic_loss": critic_loss}
                if actor_loss is not None:
                    logs["train/actor_loss"] = actor_loss
                wandb.log(logs, step=t)

        if done:
            wandb.log({
                "train/episode_reward": episode_reward,
                "train/episode_length": episode_timesteps
            }, step=t)

            print(f"Step {t} | Episode {episode_num} | Reward: {episode_reward:.2f}")

            state, _ = env.reset()
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        if (t + 1) % conf.eval_freq == 0:
            print(f"Evaluating at step {t+1}...")
            mean_eval_score, video_path = evaluate_and_record(policy, conf.env_id, conf.seed, step=t+1)

            wandb.log({"eval/mean_reward": mean_eval_score}, step=t)

            if video_path:
                print(f"Uploading video: {video_path}")
                wandb.log({"eval/video": wandb.Video(video_path, fps=30, format="mp4")}, step=t)

    print("Training Complete.")

    if conf.save_model:
        save_path = "td3_model_artifacts"
        os.makedirs(save_path, exist_ok=True)

        policy.save(os.path.join(save_path, "td3_lunarlander"))

        readme = f"""
---
tags:
- deep-reinforcement-learning
- reinforcement-learning
- gymnasium
- td3
library_name: stable-baselines3
env_id: {conf.env_id}
model-index:
- name: TD3
  results:
  - task:
      type: reinforcement-learning
      name: reinforcement-learning
    dataset:
      name: {conf.env_id}
      type: {conf.env_id}
    metrics:
    - type: mean_reward
      value: {mean_eval_score:.2f}
      name: mean_reward
---

# TD3 Agent for {conf.env_id}

This is a **Twin Delayed DDPG (TD3)** agent trained on `{conf.env_id}`.

## Experiment Results
- **Final Mean Reward:** {mean_eval_score:.2f}
- **Total Timesteps:** {conf.total_timesteps}

## Hyperparameters
```python
{asdict(conf)}


"""
    with open(os.path.join(save_path, "README.md"), "w") as f:
        f.write(readme)

    try:
        print(f"Uploading model to Hugging Face Hub: {conf.hf_repo_id}")
        api = HfApi()
        create_repo(repo_id=conf.hf_repo_id, exist_ok=True, repo_type="model")
        upload_folder(
            folder_path=save_path,
            repo_id=conf.hf_repo_id,
            repo_type="model",
            commit_message="Upload TD3 agent with video and charts logs"
        )
        print("Upload success!")
    except Exception as e:
        print(f"HF Upload failed (check your token): {e}")

wandb.finish()

run_training()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 409c26fd421dbaf2466bc661106ac4e6d9a8ae97


wandb: WARNING Invalid choice
wandb: Enter your choice:

 409c26fd421dbaf2466bc661106ac4e6d9a8ae97


wandb: WARNING Invalid choice
wandb: Enter your choice:

...

Uploading video: videos/td3-step-90000-episode-0.mp4
Step 90369 | Episode 298 | Reward: 244.55
Step 90884 | Episode 299 | Reward: 221.44
Step 91381 | Episode 300 | Reward: 227.49
Step 92060 | Episode 301 | Reward: 196.72
Step 92595 | Episode 302 | Reward: 191.68
Step 93001 | Episode 303 | Reward: 193.98
Step 93976 | Episode 304 | Reward: 168.10
Step 94341 | Episode 305 | Reward: -43.16
Step 94720 | Episode 306 | Reward: 215.33
Evaluating at step 95000...


wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


Uploading video: videos/td3-step-95000-episode-0.mp4
Step 95411 | Episode 307 | Reward: 194.96
Step 96019 | Episode 308 | Reward: 227.12
Step 96678 | Episode 309 | Reward: -187.17
Step 97083 | Episode 310 | Reward: 222.64
Step 97462 | Episode 311 | Reward: 250.36
Step 98168 | Episode 312 | Reward: 147.79
Step 98503 | Episode 313 | Reward: 220.61
Step 98873 | Episode 314 | Reward: 240.02
Step 99246 | Episode 315 | Reward: 238.02
Step 99775 | Episode 316 | Reward: 198.46
Evaluating at step 100000...


wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


Uploading video: videos/td3-step-100000-episode-0.mp4
Training Complete.
Uploading model to Hugging Face Hub: yousefyousefyousef335/td3-lunarlander-v3
HF Upload failed (check your token): (Request ID: Root=1-6939c45f-5862bcb4438d0b110f9c7ad0;e6650954-d85f-4e6a-925c-f409673039e8)

403 Forbidden: You don't have the rights to create a model under the namespace "yousefyousefyousef335".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.


In [8]:

api = HfApi()
create_repo(repo_id="YoussefTolba/td3-lunarlander-v3", exist_ok=True, repo_type="model")
upload_folder(
    folder_path="td3_model_artifacts",
    repo_id="YoussefTolba/td3-lunarlander-v3",
    repo_type="model",
    commit_message="Upload TD3 agent with video and charts logs"
)
print("Upload success!")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...td3_lunarlander_actor.pth:  66%|######5   |  183kB /  278kB            

  ...d3_lunarlander_critic.pth:  66%|######5   |  367kB /  556kB            

Upload success!


##  CarRacing-v3

In [1]:
import os
import random
import time
import uuid
from dataclasses import dataclass, asdict
from typing import Optional, Tuple

# --- KAGGLE SPECIFIC: START VIRTUAL DISPLAY ---
from pyvirtualdisplay import Display
try:
    # Create a virtual screen to trick OpenGL
    display = Display(visible=0, size=(1400, 900))
    display.start()
    print("Virtual display started successfully.")
except Exception as e:
    print(f"Failed to start virtual display: {e}")
# ---------------------------------------------

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from huggingface_hub import HfApi, create_repo, upload_folder
from kaggle_secrets import UserSecretsClient # For secure Auth

# ==========================================
# 0. Setup WandB Login for Kaggle
# ==========================================
try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api_key")
    wandb.login(key=wandb_api_key)
    print("Logged into WandB via Kaggle Secrets.")
except:
    print("Could not find 'wandb_api_key' in Kaggle Secrets. Falling back to interactive login.")
    wandb.login()

# ==========================================
# 1. Configuration & Hyperparameters
# ==========================================
@dataclass
class TD3Config:
    # Experiment Settings
    env_id: str = "CarRacing-v3"
    project_name: str = "td3-carracing6"
    run_name: str = f"td3_car_{str(uuid.uuid4())[:8]}"
    seed: int = 42

    # Training Duration
    total_timesteps: int = 500_000
    moving_forwards_timsteps: int = 2000
    learning_starts: int = 10_000

    # Hyperparameters
    hidden_dim: int = 256
    actor_lr: float = 1e-4
    critic_lr: float = 1e-4
    batch_size: int = 64
    buffer_size: int = 100_000
    gamma: float = 0.99
    tau: float = 0.005

    # TD3 Specifics
    policy_noise: float = 0.2
    noise_clip: float = 0.5
    policy_delay: int = 2
    exploration_noise: float = 0.1

    # Logging & Saving
    eval_freq: int = 10_000
    save_model: bool = True
    hf_repo_id: str = "yousefyousefyousef335/td3-carracing-v3"

# ==========================================
# 2. Preprocessing & Wrappers
# ==========================================
class ImageTransposeWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255,
            shape=(obs_shape[2], obs_shape[0], obs_shape[1]),
            dtype=np.uint8
        )

    def observation(self, observation):
        return np.transpose(observation, (2, 0, 1))

# ==========================================
# 3. Replay Buffer (Optimized for Images)
# ==========================================
class ReplayBuffer:
    def __init__(self, state_shape, action_dim, max_size=1e5):
        self.max_size = int(max_size)
        self.ptr = 0
        self.size = 0
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.state = np.zeros((self.max_size, *state_shape), dtype=np.uint8)
        self.action = np.zeros((self.max_size, action_dim), dtype=np.float32)
        self.next_state = np.zeros((self.max_size, *state_shape), dtype=np.uint8)
        self.reward = np.zeros((self.max_size, 1), dtype=np.float32)
        self.not_done = np.zeros((self.max_size, 1), dtype=np.float32)

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)

        return (
            torch.FloatTensor(self.state[ind]).to(self.device) / 255.0,
            torch.FloatTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind]).to(self.device) / 255.0,
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )

# ==========================================
# 4. Neural Networks (CNN + MLP)
# ==========================================
class CNNEncoder(nn.Module):
    def __init__(self, input_channels=3):
        super(CNNEncoder, self).__init__()
        # Input: (3, 96, 96)
        self.net = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=8, stride=4), # -> (32, 23, 23)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),             # -> (64, 10, 10)
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),             # -> (64, 8, 8)
            nn.ReLU(),
            nn.Flatten()
        )
        self.out_dim = 64 * 8 * 8

    def forward(self, x):
        return self.net(x)

class Actor(nn.Module):
    def __init__(self, action_dim, max_action, hidden_dim=256):
        super(Actor, self).__init__()
        self.encoder = CNNEncoder()

        self.l1 = nn.Linear(self.encoder.out_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, action_dim)
        self.max_action = max_action

    def forward(self, state):
        features = self.encoder(state)
        a = F.relu(self.l1(features))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a))

class Critic(nn.Module):
    def __init__(self, action_dim, hidden_dim=256):
        super(Critic, self).__init__()
        self.encoder1 = CNNEncoder()
        self.encoder2 = CNNEncoder()

        # Q1 Architecture
        self.l1 = nn.Linear(self.encoder1.out_dim + action_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, 1)

        # Q2 Architecture
        self.l4 = nn.Linear(self.encoder2.out_dim + action_dim, hidden_dim)
        self.l5 = nn.Linear(hidden_dim, hidden_dim)
        self.l6 = nn.Linear(hidden_dim, 1)

    def forward(self, state, action):
        f1 = self.encoder1(state)
        sa1 = torch.cat([f1, action], 1)
        q1 = F.relu(self.l1(sa1))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        f2 = self.encoder2(state)
        sa2 = torch.cat([f2, action], 1)
        q2 = F.relu(self.l4(sa2))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        f1 = self.encoder1(state)
        sa = torch.cat([f1, action], 1)
        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

# ==========================================
# 5. TD3 Algorithm
# ==========================================
class TD3:
    def __init__(self, action_dim, max_action, config: TD3Config):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conf = config
        self.max_action = max_action
        self.total_it = 0

        self.actor = Actor(action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target = Actor(action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=config.actor_lr)

        self.critic = Critic(action_dim, config.hidden_dim).to(self.device)
        self.critic_target = Critic(action_dim, config.hidden_dim).to(self.device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=config.critic_lr)

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device) / 255.0
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer):
        self.total_it += 1

        state, action, next_state, reward, not_done = replay_buffer.sample(self.conf.batch_size)

        with torch.no_grad():
            noise = (torch.randn_like(action) * self.conf.policy_noise).clamp(-self.conf.noise_clip, self.conf.noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + not_done * self.conf.gamma * target_Q

        current_Q1, current_Q2 = self.critic(state, action)
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        actor_loss = None
        if self.total_it % self.conf.policy_delay == 0:
            actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)
            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)

        return critic_loss.item(), (actor_loss.item() if actor_loss else None)

    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic.pth")
        torch.save(self.actor.state_dict(), filename + "_actor.pth")

# ==========================================
# 6. Helpers: Evaluation & Recording (UPDATED)
# ==========================================
def evaluate_and_record(policy, env_id, seed, step, run_name, video_folder="videos"):
    """
    Evaluates the policy and saves the video with a unique structure to avoid overwriting.
    """
    # 1. Create a unique folder for THIS specific run
    run_video_folder = os.path.join(video_folder, run_name)
    os.makedirs(run_video_folder, exist_ok=True)

    eval_env = gym.make(env_id, continuous=True, render_mode="rgb_array")
    eval_env = ImageTransposeWrapper(eval_env)

    # 2. Define a unique prefix using the step count
    video_prefix = f"step-{step}"

    eval_env = gym.wrappers.RecordVideo(
        eval_env,
        video_folder=run_video_folder,
        name_prefix=video_prefix,
        episode_trigger=lambda x: True,
        disable_logger=True
    )

    rewards = []
    video_path = None

    # 1 Episode Eval
    for i in range(1):
        state, _ = eval_env.reset(seed=seed + 100 + i)
        terminated, truncated = False, False
        episode_reward = 0

        while not (terminated or truncated):
            action = policy.select_action(np.array(state))
            state, reward, terminated, truncated, _ = eval_env.step(action)
            episode_reward += reward

        rewards.append(episode_reward)

    eval_env.close()

    # Construct the expected path to verify it exists
    expected_filename = f"{video_prefix}-episode-0.mp4"
    expected_path = os.path.join(run_video_folder, expected_filename)

    if os.path.exists(expected_path):
        video_path = expected_path

    return rewards, video_path

# ==========================================
# 7. Main Training Loop
# ==========================================
def run_training():
    conf = TD3Config()

    wandb.init(
        project=conf.project_name,
        name=conf.run_name,
        config=asdict(conf),
        monitor_gym=False,
        save_code=True
    )

    env = gym.make(conf.env_id, continuous=True)
    env = ImageTransposeWrapper(env)

    env.action_space.seed(conf.seed)
    torch.manual_seed(conf.seed)
    np.random.seed(conf.seed)

    state_shape = env.observation_space.shape
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])

    replay_buffer = ReplayBuffer(state_shape, action_dim, conf.buffer_size)
    policy = TD3(action_dim, max_action, conf)

    state, _ = env.reset(seed=conf.seed)
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0

    print(f"---------------------------------------")
    print(f"Starting Training: {conf.env_id} | TD3 | Seed: {conf.seed}")
    print(f"Observation Shape: {state_shape} | Action Dim: {action_dim}")
    print(f"---------------------------------------")

    for t in range(int(conf.total_timesteps)):
        episode_timesteps += 1

       if t < conf.learning_starts:
            action = env.action_space.sample()

        else:
            raw_action = policy.select_action(np.array(state))
            noise = np.random.normal(0, max_action * conf.exploration_noise, size=action_dim)

            # Standard clipping to environment bounds [-1, 1]
            action = (raw_action + noise).clip(-max_action, max_action)

            # >>> CUSTOM CLIPPING: First 100k Steps <<<
            if t < 100_000:
                # Steering: [-0.25, 0.25]
                action[0] = np.clip(action[0], -0.01, 0.01)
                # Gas: [0.1, 1.0] (Forces agent to keep moving)
                action[1] = np.clip(action[1], 0.1, 1.0)
                # Brake: [0.0, 0.8] (Prevents full hard braking)
                action[2] = np.clip(action[2], 0.0, 0.8)
            # >>> END CUSTOM CLIPPING <<<

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Store data in replay buffer
        replay_buffer.add(state, action, next_state, reward, float(terminated))
        state = next_state
        episode_reward += reward

        if t >= conf.learning_starts:
            critic_loss, actor_loss = policy.train(replay_buffer)

            if t % 100 == 0:
                logs = {"train/critic_loss": critic_loss}
                if actor_loss is not None:
                    logs["train/actor_loss"] = actor_loss
                wandb.log(logs, step=t)

        if done:
            wandb.log({
                "train/episode_reward": episode_reward,
                "train/episode_length": episode_timesteps
            }, step=t)

            print(f"Step {t} | Episode {episode_num} | Reward: {episode_reward:.2f}")

            state, _ = env.reset()
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        if (t + 1) % conf.eval_freq == 0:
            print(f"Evaluating at step {t+1}...")

            eval_rewards, video_path = evaluate_and_record(
                policy,
                conf.env_id,
                conf.seed,
                step=t+1,
                run_name=conf.run_name
            )

            mean_score = np.mean(eval_rewards)
            wandb.log({"eval/mean_reward": mean_score}, step=t)
            wandb.log({"eval/episode_reward": eval_rewards[0]}, step=t)

            if video_path:
                print(f"Uploading video: {video_path}")
                wandb.log({
                    "eval/video": wandb.Video(
                        video_path,
                        fps=30,
                        format="mp4",
                        caption=f"Eval Step {t+1} | Score: {mean_score:.2f}"
                    )
                }, step=t)

    print("Training Complete.")

    if conf.save_model:
        save_path = "td3_model_artifacts"
        os.makedirs(save_path, exist_ok=True)
        policy.save(os.path.join(save_path, "td3_carracing"))

    wandb.finish()

if __name__ == "__main__":
    run_training()

KeyboardInterrupt: 

## enforcing car to learn the true path

In [ ]:
import os
import random
import time
import uuid
from dataclasses import dataclass, asdict
from typing import Optional, Tuple

# --- KAGGLE SPECIFIC: START VIRTUAL DISPLAY ---
from pyvirtualdisplay import Display
try:
    # Create a virtual screen to trick OpenGL
    display = Display(visible=0, size=(1400, 900))
    display.start()
    print("Virtual display started successfully.")
except Exception as e:
    print(f"Failed to start virtual display: {e}")
# ---------------------------------------------

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from huggingface_hub import HfApi, create_repo, upload_folder
from kaggle_secrets import UserSecretsClient # For secure Auth

# ==========================================
# 0. Setup WandB Login for Kaggle
# ==========================================
try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api_key")
    wandb.login(key=wandb_api_key)
    print("Logged into WandB via Kaggle Secrets.")
except:
    print("Could not find 'wandb_api_key' in Kaggle Secrets. Falling back to interactive login.")
    wandb.login()

# ==========================================
# 1. Configuration & Hyperparameters
# ==========================================
@dataclass
class TD3Config:
    # Experiment Settings
    env_id: str = "CarRacing-v3"
    project_name: str = "td3-carracing8"
    run_name: str = f"td3_car_{str(uuid.uuid4())[:8]}"
    seed: int = 42
    
    # Training Duration 
    total_timesteps: int = 500_000  
    # warm up steps (heuristic agent)
    learning_starts: int = 30_000       
    
    # Hyperparameters
    hidden_dim: int = 256
    actor_lr: float = 1e-4  
    critic_lr: float = 1e-4
    batch_size: int = 64
    buffer_size: int = 100_000 
    gamma: float = 0.99               
    tau: float = 0.005                
    
    # TD3 Specifics
    policy_noise: float = 0.2         
    noise_clip: float = 0.5           
    policy_delay: int = 2             
    exploration_noise: float = 0.1  

    # Logging & Saving
    eval_freq: int = 10_000
    save_model: bool = True
    hf_repo_id: str = "yousefyousefyousef335/td3-carracing-v3"

# ==========================================
# 2. Preprocessing & Wrappers
# ==========================================
class ImageTransposeWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255, 
            shape=(obs_shape[2], obs_shape[0], obs_shape[1]), 
            dtype=np.uint8
        )

    def observation(self, observation):
        return np.transpose(observation, (2, 0, 1))

# ==========================================
# 3. Replay Buffer (Optimized for Images)
# ==========================================
class ReplayBuffer:
    def __init__(self, state_shape, action_dim, max_size=1e5):
        self.max_size = int(max_size)
        self.ptr = 0
        self.size = 0
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.state = np.zeros((self.max_size, *state_shape), dtype=np.uint8)
        self.action = np.zeros((self.max_size, action_dim), dtype=np.float32)
        self.next_state = np.zeros((self.max_size, *state_shape), dtype=np.uint8)
        self.reward = np.zeros((self.max_size, 1), dtype=np.float32)
        self.not_done = np.zeros((self.max_size, 1), dtype=np.float32)

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        
        return (
            torch.FloatTensor(self.state[ind]).to(self.device) / 255.0,
            torch.FloatTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind]).to(self.device) / 255.0,
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )

# ==========================================
# 4. Neural Networks (CNN + MLP)
# ==========================================
class CNNEncoder(nn.Module):
    def __init__(self, input_channels=3):
        super(CNNEncoder, self).__init__()
        # Input: (3, 96, 96)
        self.net = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=8, stride=4), # -> (32, 23, 23)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),             # -> (64, 10, 10)
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),             # -> (64, 8, 8)
            nn.ReLU(),
            nn.Flatten()
        )
        self.out_dim = 64 * 8 * 8 

    def forward(self, x):
        return self.net(x)

class Actor(nn.Module):
    def __init__(self, action_dim, max_action, hidden_dim=256):
        super(Actor, self).__init__()
        self.encoder = CNNEncoder()
        
        self.l1 = nn.Linear(self.encoder.out_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, action_dim)
        self.max_action = max_action

    def forward(self, state):
        features = self.encoder(state)
        a = F.relu(self.l1(features))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a)) 

class Critic(nn.Module):
    def __init__(self, action_dim, hidden_dim=256):
        super(Critic, self).__init__()
        self.encoder1 = CNNEncoder()
        self.encoder2 = CNNEncoder()

        # Q1 Architecture
        self.l1 = nn.Linear(self.encoder1.out_dim + action_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, 1)

        # Q2 Architecture
        self.l4 = nn.Linear(self.encoder2.out_dim + action_dim, hidden_dim)
        self.l5 = nn.Linear(hidden_dim, hidden_dim)
        self.l6 = nn.Linear(hidden_dim, 1)

    def forward(self, state, action):
        f1 = self.encoder1(state)
        sa1 = torch.cat([f1, action], 1)
        q1 = F.relu(self.l1(sa1))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        f2 = self.encoder2(state)
        sa2 = torch.cat([f2, action], 1)
        q2 = F.relu(self.l4(sa2))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        f1 = self.encoder1(state)
        sa = torch.cat([f1, action], 1)
        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

# ==========================================
# 5. TD3 Algorithm
# ==========================================
class TD3:
    def __init__(self, action_dim, max_action, config: TD3Config):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conf = config
        self.max_action = max_action
        self.total_it = 0

        self.actor = Actor(action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target = Actor(action_dim, max_action, config.hidden_dim).to(self.device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=config.actor_lr)

        self.critic = Critic(action_dim, config.hidden_dim).to(self.device)
        self.critic_target = Critic(action_dim, config.hidden_dim).to(self.device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=config.critic_lr)

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device) / 255.0
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer):
        self.total_it += 1
        
        state, action, next_state, reward, not_done = replay_buffer.sample(self.conf.batch_size)

        with torch.no_grad():
            noise = (torch.randn_like(action) * self.conf.policy_noise).clamp(-self.conf.noise_clip, self.conf.noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + not_done * self.conf.gamma * target_Q

        current_Q1, current_Q2 = self.critic(state, action)
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        actor_loss = None
        if self.total_it % self.conf.policy_delay == 0:
            actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
            
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)
            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.conf.tau * param.data + (1 - self.conf.tau) * target_param.data)

        return critic_loss.item(), (actor_loss.item() if actor_loss else None)

    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic.pth")
        torch.save(self.actor.state_dict(), filename + "_actor.pth")

# ==========================================
# 6. EXPERT HEURISTIC AGENT
# ==========================================
class CarRacingHeuristic:
    def __init__(self):
        # We target a modest speed for safe "expert" data collection
        self.target_speed = 0.1 
        self.k_p = 1.8  # Proportional gain for steering

    def act(self, state):
        """
        Input state: (3, 96, 96) in [0..255]
        Returns: action [steering, gas, brake]
        """
        # Ensure numpy
        if isinstance(state, torch.Tensor):
            state = state.cpu().numpy()

        # Convert back to HWC for image processing (96, 96, 3)
        img = np.transpose(state, (1, 2, 0))
        
        # Look ahead crop (approx rows 60-78)
        crop = img[60:78, :, :]
        
        # Simple road detection: Road is gray (R ~= G ~= B)
        r, g, b = crop[:, :, 0], crop[:, :, 1], crop[:, :, 2]
        
        # Road is where colors are close to each other, and not too dark
        is_road = (np.abs(r - g) < 15) & (np.abs(g - b) < 15) & (g > 60)
        
        road_pixels = np.argwhere(is_road)
        
        if len(road_pixels) > 0:
            # Average column index (axis 1) is the center of the road
            target_x = np.mean(road_pixels[:, 1])
            
            # Center of the image is 48
            error = (target_x - 48.0) / 48.0
            
            steering = np.clip(error * self.k_p, -1.0, 1.0)
            
            # Slow down on sharp turns
            gas = 0.05 if abs(steering) > 0.3 else 0.2
            brake = 0.0
        else:
            # If lost, usually better to do nothing or slight brake than spin
            steering = 0.0
            gas = 0.0
            brake = 0.1

        return np.array([steering, gas, brake], dtype=np.float32)

# ==========================================
# 7. Helpers: Evaluation & Recording
# ==========================================
def evaluate_and_record(policy, env_id, seed, step, run_name, video_folder="videos"):
    run_video_folder = os.path.join(video_folder, run_name)
    os.makedirs(run_video_folder, exist_ok=True)
    
    eval_env = gym.make(env_id, continuous=True, render_mode="rgb_array")
    eval_env = ImageTransposeWrapper(eval_env)
    
    video_prefix = f"step-{step}"
    
    eval_env = gym.wrappers.RecordVideo(
        eval_env, 
        video_folder=run_video_folder, 
        name_prefix=video_prefix,
        episode_trigger=lambda x: True, 
        disable_logger=True
    )
    
    rewards = []
    video_path = None
    
    for i in range(1): 
        state, _ = eval_env.reset(seed=seed + 100 + i)
        terminated, truncated = False, False
        episode_reward = 0
        
        while not (terminated or truncated):
            action = policy.select_action(np.array(state))
            state, reward, terminated, truncated, _ = eval_env.step(action)
            episode_reward += reward
            
        rewards.append(episode_reward)
    
    eval_env.close()
    
    expected_filename = f"{video_prefix}-episode-0.mp4"
    expected_path = os.path.join(run_video_folder, expected_filename)
    
    if os.path.exists(expected_path):
        video_path = expected_path
        
    return rewards, video_path

# ==========================================
# 8. Main Training Loop
# ==========================================
def run_training():
    conf = TD3Config()
    
    wandb.init(
        project=conf.project_name,
        name=conf.run_name,
        config=asdict(conf),
        monitor_gym=False,
        save_code=True
    )

    env = gym.make(conf.env_id, continuous=True)
    env = ImageTransposeWrapper(env)
    
    env.action_space.seed(conf.seed)
    torch.manual_seed(conf.seed)
    np.random.seed(conf.seed)

    state_shape = env.observation_space.shape
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])

    replay_buffer = ReplayBuffer(state_shape, action_dim, conf.buffer_size)
    policy = TD3(action_dim, max_action, conf)
    
    # Initialize heuristic expert
    expert = CarRacingHeuristic()

    state, _ = env.reset(seed=conf.seed)
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0

    print(f"---------------------------------------")
    print(f"Starting Training: {conf.env_id} | TD3 | Seed: {conf.seed}")
    print(f"Observation Shape: {state_shape} | Action Dim: {action_dim}")
    print(f"Warmup (Expert) Steps: {conf.learning_starts}")
    print(f"---------------------------------------")

    for t in range(int(conf.total_timesteps)):
        episode_timesteps += 1

        # === 1. Warmup Phase (Expert Data Collection) ===
        if t < conf.learning_starts:
            # Use heuristic expert
            action = expert.act(state)
            
            # Add small noise so the data isn't identical (Robustness)
            action[0] += np.random.normal(0, 0.1) # Steering noise
            action[0] = np.clip(action[0], -1.0, 1.0)
            
            # Ensure Gas/Brake are valid [0,1]
            action[1] = np.clip(action[1], 0.0, 1.0)
            action[2] = np.clip(action[2], 0.0, 1.0)

        # === 2. Training Phase (TD3 Policy) ===
        else:
            raw_action = policy.select_action(np.array(state))
            noise = np.random.normal(0, max_action * conf.exploration_noise, size=action_dim)
            
            action = (raw_action + noise).clip(-max_action, max_action)
            
            # CarRacing Env Constraints: 
            # Steering: [-1, 1], Gas: [0, 1], Brake: [0, 1]
            action[1] = max(action[1], 0.0) 
            action[2] = max(action[2], 0.0)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        # Store in buffer
        replay_buffer.add(state, action, next_state, reward, float(terminated))
        state = next_state
        episode_reward += reward

        # Train Policy
        if t >= conf.learning_starts:
            critic_loss, actor_loss = policy.train(replay_buffer)
            
            if t % 100 == 0:
                logs = {"train/critic_loss": critic_loss}
                if actor_loss is not None:
                    logs["train/actor_loss"] = actor_loss
                wandb.log(logs, step=t)

        if done:
            wandb.log({
                "train/episode_reward": episode_reward,
                "train/episode_length": episode_timesteps
            }, step=t)
            
            print(f"Step {t} | Episode {episode_num} | Reward: {episode_reward:.2f}")
            
            state, _ = env.reset()
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        # Evaluation
        if (t + 1) % conf.eval_freq == 0:
            print(f"Evaluating at step {t+1}...")
            
            eval_rewards, video_path = evaluate_and_record(
                policy, 
                conf.env_id, 
                conf.seed, 
                step=t+1,
                run_name=conf.run_name 
            )
            
            mean_score = np.mean(eval_rewards)
            wandb.log({"eval/mean_reward": mean_score}, step=t)
            wandb.log({"eval/episode_reward": eval_rewards[0]}, step=t)
            
            if video_path:
                print(f"Uploading video: {video_path}")
                wandb.log({
                    "eval/video": wandb.Video(
                        video_path, 
                        fps=30, 
                        format="mp4", 
                        caption=f"Eval Step {t+1} | Score: {mean_score:.2f}"
                    )
                }, step=t)

    print("Training Complete.")
    
    if conf.save_model:
        save_path = "td3_model_artifacts"
        os.makedirs(save_path, exist_ok=True)
        policy.save(os.path.join(save_path, "td3_carracing"))

    wandb.finish()

if __name__ == "__main__":
    run_training()